In [29]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u
#import alipy

In [30]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [31]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [32]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [33]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    if 15 < x1 < header['NAXIS1'] - 15 and 15 < y1 < header['NAXIS2'] - 15:
        try:
            mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
            AGNdata[mask] = float(0)
            mask = AGNdata == 0.
            rows = np.flatnonzero((~mask).sum(axis=1))
            cols = np.flatnonzero((~mask).sum(axis=0))
            AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
            #print apparature/float(pixspa)
        except:
            AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    else:
        AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    return AGNdata, x1, y1

In [93]:

quasJ = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/')]

#sky_RA_WORLD = 174.75445032458904 #Random patch of sky
#sky_DEC_WORLD = -37.77465736280882

#sky_RA_WORLD = SkyCoord('22:09:27 -47:07:59', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
#sky_DEC_WORLD = SkyCoord('22:09:27 -47:07:59', unit=(u.hourangle, u.deg)).dec.degree #NGC7213

sky_RA_WORLD = SkyCoord('15:30:35 +06:02:12', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
sky_DEC_WORLD = SkyCoord('15:30:35 +06:02:12', unit=(u.hourangle, u.deg)).dec.degree 
#22:09:06 -47:09:13

In [94]:
for i in range(len(quasJ)): 
    print i, quasJ[i]
    data1, header = fits.getdata(quasJ[i],header=True) #RA---TAN data2 = data1
    data1 = np.array(data1,dtype='float')
    data2, header2 = fits.getdata(quasJ[i],header=True)
    data2 = np.array(data2,dtype='float')
    #print data1
    #print header['CTYPE2']
    #header['CTYPE1'] = "RA---TAN -SIP"
    #header['CTYPE2'] = "DEC--TAN -SIP"
    #print data1
    back = np.ones((header['NAXIS1'],header['NAXIS2']))
    a_sky = curve(data1,header,WCS(quasJ[i]),
                   (sky_RA_WORLD,sky_DEC_WORLD),0.001)[0]
    a_sky_mean = a_sky[a_sky!=0].mean()
    back[data2 == 0] *= 0
    back[data2 != 0] *= a_sky_mean
    #print a_sky_mean
    #data1 = np.subtract(data1,back)
    #print data1
    data2 = data2 - back
    #print data2
    #print data2
    #for j in range(len(np.shape(data[0]))):
    #    for k in range(len(np.shape(data[0]))):
    #        if data[j,k] != 0:
    #            data[j,k] = float(data[j,k]) - a_sky_mean
    #header['NDIT'] = 5
    '''header['A_ORDER'] = "0"
    header['B_ORDER'] = "0"
    header['AP_ORDER'] = "0"
    header['BP_ORDER'] = "0"
    header['A_0_2'] = "0"
    header['A_1_1'] = "0"
    header['A_2_0'] = "0"
    header['B_0_2'] = "0"
    header['B_1_1'] = "0"
    header['B_2_0'] = "0"
    header['AP_0_2'] = "0"
    header['AP_1_1'] = "0"
    header['AP_2_0'] = "0"
    header['AP_0_1'] = "0"
    header['AP_1_0'] = "0"
    header['BP_0_1'] = "0"
    header['BP_0_2'] = "0"
    header['BP_1_0'] = "0"
    header['BP_1_1'] = "0"
    header['BP_2_0'] = "0"'''
    fits.writeto(quasJ[i],data2,header,clobber=True)
    #print fits.getheader(quasH[i])['A_ORDER']

0 /media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/IMG2017170BLh009.fits
1 /media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/IMG2017170BLh010.fits
2 /media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/IMG2017170BLh013.fits
3 /media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/IMG2017170BLh014.fits
4 /media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/IMG2017170BLs011.fits
5 /media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/IMG2017170BLs012.fits
6 /media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/IMG2017170BLs015.fits
7 /media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/IMG2017170BLs016.fits
